In [1]:
import os
import pandas as pd

In [2]:
os.chdir("/mnt/hgfs/G/Cell_line/HCT116")
os.listdir()

[]

In [3]:
def RunCmd(cmd):
    print(cmd)
    os.system(cmd)
    
# netMHCPan Predicting
def AffinityPredicting(df_peptides, prefix, HLAinfo, outdir):
    netMHCPan = "/home/guanxiangyu/netMHCpan-4.1/netMHCpan"
    os.makedirs(outdir, exist_ok=True)
    
    # HLA type included in netMHCPan
    netMHCPan_included_HLA = list()
    tmp = os.popen("{} -listMHC".format(netMHCPan))
    for line in tmp:
        line = line.strip()
        if line and (not line.startswith('#')):
            netMHCPan_included_HLA.append(line)
            
    # HLA type filtering
    if type(HLAinfo) == str and os.path.exists(HLAinfo):
        # for file input
        hla_f = ReadFile(HLAinfo)
        hla_type = hla_f.read().strip().split("\n")
        hla_f.close()
    else:
        # for HLA type list or string
        hla_type = HLAinfo
    hla_type = sorted(set(hla_type))
    for hla in hla_type:
        if hla in netMHCPan_included_HLA:
            continue
        else:
            print(">>>either {} cannot be predicted by netMHCpan-4.1 or your input format could be wrong.<<<".format(hla))
            hla_type.remove(hla)
    
    # netMHCPan predicting
    pep_file = os.path.join(outdir, '{}.pep'.format(prefix))
    df_peptides.Sequence.drop_duplicates().to_csv(pep_file, header=False, index=False, sep="\t")
    cmd = "{} -p {} -BA -inptype 1 -a {} > {}.txt".format(netMHCPan, pep_file, ','.join(hla_type), pep_file)
    RunCmd(cmd)
    cmd = "rm {}".format(pep_file)
    RunCmd(cmd)
    # netMHCPan result processing
    netmhc_result_trans = "/mnt/hgfs/G/Micro_Maxquant_v2/scripts/formattingNetMHCpanResults-n.pl"
    cmd = "{} {}.txt {} {}".format(netmhc_result_trans, pep_file, outdir, prefix)
    RunCmd(cmd)

In [7]:
df_pep = pd.read_csv("HCT116_Synpep.list", header=None, sep="\t")
df_pep.columns = ['Sequence']
df_pep.head()

,Sequence
0,TSGDPPPPP
1,ENVQIIHF
2,EVVSQLVIP
3,ERQIGLISA
4,NIDLLQQF


In [8]:
HLA_l = ['HLA-A01:01', 'HLA-A02:01', 'HLA-B45:01', 'HLA-B50:01', 'HLA-B18:01', 'HLA-C05:01', 'HLA-C07:01',]

In [9]:
os.getcwd()

'/mnt/hgfs/G/Cell_line/HCT116'

In [22]:
AffinityPredicting(df_pep, 'HCT116_Syn', HLA_l, os.getcwd())

/home/guanxiangyu/netMHCpan-4.1/netMHCpan -p /mnt/hgfs/G/Cell_line/HCT116/HCT116_Syn.pep -BA -inptype 1 -a HLA-A01:01,HLA-A02:01,HLA-B18:01,HLA-B45:01,HLA-B50:01,HLA-C05:01,HLA-C07:01 > /mnt/hgfs/G/Cell_line/HCT116/HCT116_Syn.pep.txt
rm /mnt/hgfs/G/Cell_line/HCT116/HCT116_Syn.pep
/mnt/hgfs/G/Micro_Maxquant_v2/scripts/formattingNetMHCpanResults-n.pl /mnt/hgfs/G/Cell_line/HCT116/HCT116_Syn.pep.txt /mnt/hgfs/G/Cell_line/HCT116 HCT116_Syn


In [23]:
df_NetMHC_result = pd.read_csv("HCT116_Syn.peptides.affinity_features.txt", header=0, sep="\t")
df_NetMHC_result.head()

,#Peptide,Length,BinderNum,MHC,Aff.IC50nM,Rank_EL(%),Rank_BA(%),BinderType
0,DLDEILHF,8,1,"HLA-A*01:01,HLA-A*02:01,HLA-B*18:01,HLA-B*45:0...","18544.11,30731.04,21632.20,42808.53,43746.41,1...","2.308,10.053,3.825,20.364,21.367,0.875,10.380","7.792,44.701,13.143,79.718,88.209,5.325,48.921","NB,NB,NB,NB,NB,WB,NB"
1,EDRAQLAI,8,0,"HLA-A*01:01,HLA-A*02:01,HLA-B*18:01,HLA-B*45:0...","33710.33,41865.85,26642.18,21839.62,28421.45,3...","31.750,59.545,9.917,7.258,12.582,17.000,55.000","40.107,79.691,20.596,10.497,25.088,21.899,73.214","NB,NB,NB,NB,NB,NB,NB"
2,ENIVSFNF,8,1,"HLA-A*01:01,HLA-A*02:01,HLA-B*18:01,HLA-B*45:0...","34605.77,42505.78,2764.49,19737.44,29803.37,41...","29.379,78.333,1.788,12.524,29.316,35.800,26.714","43.940,82.225,1.213,8.994,28.466,55.277,42.796","NB,NB,WB,NB,NB,NB,NB"
3,ENVQIIHF,8,1,"HLA-A*01:01,HLA-A*02:01,HLA-B*18:01,HLA-B*45:0...","38575.97,44794.88,8363.00,29837.89,36691.08,42...","22.842,63.750,1.224,7.064,21.245,27.077,14.909","63.886,90.863,3.490,20.607,53.205,62.480,48.253","NB,NB,WB,NB,NB,NB,NB"
4,ERQIGLISA,9,0,"HLA-A*01:01,HLA-A*02:01,HLA-B*18:01,HLA-B*45:0...","40938.61,37962.76,26649.96,15393.31,13863.84,4...","52.742,43.143,11.958,4.558,2.913,46.500,8.142","77.034,65.169,20.611,6.495,6.358,73.149,11.504","NB,NB,NB,NB,NB,NB,NB"


In [24]:
for i in df_NetMHC_result.columns[3:]:
    df_NetMHC_result[i] = df_NetMHC_result[i].str.split(',')

In [25]:
df_NetMHC_result.explode(df_NetMHC_result.columns[3:].to_list(), ignore_index=True).to_csv("HCT116_Syn.peptides.affinity_features.csv", index=False)

In [19]:
df_NetMHC_result.columns[3:].to_list()

['MHC', 'Aff.IC50nM', 'Rank_EL(%)', 'Rank_BA(%)', 'BinderType']